<a href="https://colab.research.google.com/github/ziyadamz2/AutomatisationMail/blob/main/slm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import pandas as pd
import re
import json
from datasets import load_dataset
pd.set_option('display.max_colwidth', None)
from transformers import BertTokenizerFast

In [ ]:
# Dataset Question / Anwswers
dataset = load_dataset("CATIE-AQ/frenchQA")

In [ ]:
dataset

In [ ]:

def convert_to_jsonl(split, output_file):
    with open(output_file, "w", encoding="utf-8") as f:
        for ex in dataset[split]:
            context = ex["context"].strip()
            question = ex["question"].strip()
            answers = ex["answers"]["text"]
            answer = answers[0] if answers else ""

            # Nouveau format input/output
            input_text = f"Contexte : {context}\nQuestion : {question}"
            output_text = answer

            f.write(json.dumps({"input": input_text, "output": output_text}, ensure_ascii=False) + "\n")

# Sauvegarde avec les mêmes noms de fichiers
convert_to_jsonl("train", "train_concat.jsonl")
convert_to_jsonl("validation", "validation_concat.jsonl")
convert_to_jsonl("test", "test_concat.jsonl")

print("✅ Fichiers créés : train_concat.jsonl, validation_concat.jsonl, test_concat.jsonl (input/output)")


✅ Fichiers créés : train_concat.jsonl, validation_concat.jsonl, test_concat.jsonl (input/output)


Tokenniser

In [ ]:

train_dataset = load_dataset("json", data_files="/content/train_concat.jsonl")


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(train_dataset)

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 200617
    })
})


In [ ]:
# Charger le tokenizer (ici français)
tokenizer = BertTokenizerFast.from_pretrained("camembert-base")

SEQ_LEN = 256  # longueur max par séquence

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CamembertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


In [ ]:

def tokenize_function(examples):
    # Concaténer input + output en une seule séquence
    full_texts = [
        f"{inp}\nRéponse : {out}"
        for inp, out in zip(examples["input"], examples["output"])
    ]

    # Tokenisation
    return tokenizer(
        full_texts,
        padding="max_length",
        truncation=True,
        max_length=512
    )

# Appliquer sur le dataset
tokenized_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["input", "output"])
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])


Map:   0%|          | 0/200617 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_dataset['train'][0])

{'input_ids': tensor([    5, 30971,    43, 21655,    88,  2162,  2722,    10,   144,   482,
         1142,   640,   408,    26,  7335,   809,    38,   122,  2008,     3,
          603,     3,  2306,    35,     3,   268,  2446,    35,    26,  1076,
         3731,    26,    10,  2354,    53,    38,  2304,    16,   181,   652,
        11374,    53,    30,    28,  7184,     7, 19204,   233,  5479,     7,
        27098,     8,  8313,    14,  9809,  2606,     9, 23845,    14,  5544,
           15, 30729,     7,    36, 17814,     7,   109,    33,  2478,    29,
         1448,  3957,    14,    29,    35,   851,     7,    14,    30,  1758,
         1884,    15,    13,   259,    20,   318,  7863,    22,   376,    27,
         7184,    25,   368,     8,  1134,   345,   391,   385,   363,  1951,
          105,    11,    10,  4049,  8163,     9,   361,  3478,    37,    58,
          819,     7, 12456,    35,   640,   482,  1142,   640,   408,     7,
           16,   368,    30,  1758,    17,    11, 

In [ ]:
def add_labels(examples):
    examples["labels"] = examples["input_ids"].clone()
    return examples

tokenized_dataset = tokenized_dataset.map(add_labels)

Map:   0%|          | 0/200617 [00:00<?, ? examples/s]

In [ ]:
print(tokenizer.vocab_size)

32005


In [ ]:
print(tokenized_dataset['train'])
print(tokenized_dataset['train'][0])

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 200617
})
{'input_ids': tensor([    5, 30971,    43, 21655,    88,  2162,  2722,    10,   144,   482,
         1142,   640,   408,    26,  7335,   809,    38,   122,  2008,     3,
          603,     3,  2306,    35,     3,   268,  2446,    35,    26,  1076,
         3731,    26,    10,  2354,    53,    38,  2304,    16,   181,   652,
        11374,    53,    30,    28,  7184,     7, 19204,   233,  5479,     7,
        27098,     8,  8313,    14,  9809,  2606,     9, 23845,    14,  5544,
           15, 30729,     7,    36, 17814,     7,   109,    33,  2478,    29,
         1448,  3957,    14,    29,    35,   851,     7,    14,    30,  1758,
         1884,    15,    13,   259,    20,   318,  7863,    22,   376,    27,
         7184,    25,   368,     8,  1134,   345,   391,   385,   363,  1951,
          105,    11,    10,  4049,  8163,     9,   361,  3478,    37,    58,
          819,     

# Batch pour l'entrainement

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset["train"],
    batch_size=8,
    shuffle=True
)


In [ ]:
print("Max token ID:", max(tokenized_dataset["train"]["input_ids"][0]))

Max token ID: tensor(30971)


In [ ]:
import torch
import torch.nn as nn

class SimpleGPT(nn.Module):
    def __init__(self, vocab_size, d_model=128, n_heads=4, n_layers=2, max_len=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Embedding(max_len, d_model)

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_heads)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, input_ids, attention_mask=None):
        seq_len = input_ids.size(1)
        positions = torch.arange(0, seq_len, device=input_ids.device).unsqueeze(0)

        x = self.embedding(input_ids) + self.pos_embedding(positions)
        x = self.transformer(x)
        logits = self.fc_out(x)
        return logits

vocab_size = 33007
model = SimpleGPT(vocab_size)


In [ ]:
import numpy as np

# Sur tout ton dataset
max_token_id = np.max([max(x) for x in tokenized_dataset["train"]["input_ids"]])
print("Token max dans dataset :", max_token_id)

Token max dans dataset : 32007


In [ ]:
from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(3):
    for batch in train_dataloader:
        input_ids = batch["input_ids"]
        labels = batch["labels"]

        optimizer.zero_grad()
        outputs = model(input_ids)

        # CrossEntropy attend (batch*seq, vocab_size) et (batch*seq,)
        loss = loss_fn(outputs.view(-1, vocab_size), labels.view(-1))
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} - Loss: {loss.item():.4f}")

KeyboardInterrupt: 